Sometimes you don't want to use OpenRefine. Why not? I don't know, it's the *best* for cleaning up fuzzy matches. But yes, sure, sometimes maybe you don't.

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
df = pd.read_csv("CD_Transactions_07-23-2017.CSV", index_col=False)

In [ ]:
df.head()

# What are all of our options for the "Alaska Sea Pilot PAC fund"?

In [ ]:
df[df['Last/Business Name'] == 'Alaska Sea Pilot PAC fund'].shape

In [ ]:
df[df['Last/Business Name'] == 'ALASKA SEA PILOT PAC FUND'].shape

In [ ]:
df[df['Last/Business Name'] == 'Alaska Sea Pilot Pac Fund'].shape

Maybe we can throw in a regex and catch some more?

In [ ]:
df[df['Last/Business Name'].str.contains("Sea.*Pilot", na=False)]['Last/Business Name'].value_counts()

# Using fuzzywuzzy for finding fuzzy matches

Fuzzy matches are incomplete or inexact matches. The Python package [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) has a few functions that can help you, although they're a little bit confusing! I'm going to take the examples from GitHub and annotate them a little, then we'll use them.

First, install fuzzywuzzy with

```bash
pip3 install fuzzywuzzy[speedup]
```

Then we'll get to importing it

In [ ]:
# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

> **MAKE SURE YOU INSTALLED USING `pip3 install fuzzywuzzy[speedup]` OR ELSE IT WILL COMPLAIN HERE AND WILL ALSO BE SLOWER**

### `fuzz.ratio` compares the entire string, in order

Every single thing in the string is important here!

In [ ]:
fuzz.ratio("this is a test", "this is a test!")

### `fuzz.partial_ratio` compares subsections of the string

Partial matches are fine! The exclamation mark at the end made `fuzz.ratio` not like the comparison last time, but this time it's OK.

In [ ]:
fuzz.partial_ratio("this is a test", "this is a test!")

### `fuzz.token_sort_ratio` ignores word order

`fuzz.token_sort_ratio` orders all of the words first, so "KENNEDY JOHN" and "JOHN KENNEDY" would be the same.

In [ ]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

In [ ]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

### `fuzz.token_set_ratio` ignores duplicate words

I don't know why you'd ever have "JOHN KENNEDY KENNEDY" but if you use `fuzz.token_set_ratio` then it would definitely match "JOHN KENNEDY".

In [ ]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

# Actually using fuzzywuzzy on our dataset, featuring `process.extract`

Since we already imported, let's collect all of the business names into a list. We're going to search through the list to find names that are similar to **Alaska Sea Pilot PAC Fun**.

In [ ]:
# If we grab a column and use .unique(), it gives us every business name with no repeats
choices = df['Last/Business Name'].unique()
choices[:15]

Now we'll use `process.extract` to find the top 15 matches

In [ ]:
%%time
process.extract("Alaska Sea Pilot PAC Fund", choices, limit=10, scorer=fuzz.token_sort_ratio)

Wow, those look pretty nice! Maybe instead we should just find the ones that are above a certain score? You can also specify a `scorer` if you want to get particular.

In [ ]:
# Get 100 options so we're sure to have some non-matches
possibilities = process.extract("Alaska Sea Pilot PAC Fund", choices, limit=100, scorer=fuzz.token_sort_ratio)

In [ ]:
# And let's see everyting with a score above 70
[possible for possible in possibilities if possible[1] > 70]

**Huh, pretty neat.**

## Filtering directly with fuzzywuzzy

We can also use this directly with our dataframe, if we'd like to use `fuzzywuzzy` to filter instead of giving us a list. It's going to be a lot slower, but that's life, I guess.

In [ ]:
def get_ratio(row):
    name = row['Last/Business Name']
    return fuzz.token_sort_ratio(name, "Alaska Sea Pilot PAC Fund")

df[df.apply(get_ratio, axis=1) > 70]

You could also do it using a lambda if you wanted

In [ ]:
df[df.apply(lambda row: fuzz.token_sort_ratio(row['Last/Business Name'], "Alaska Sea Pilot PAC Fund"), axis=1) > 70]

## We could technically clean it like below, but... it seems risky. Because it *is* risky!

```python
df.loc[df.apply(get_ratio, axis=1) > 75, "Last/Business Name"] = "Alaska Sea Pilot PAC Fund"
```